In [0]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer


In [0]:
!pip install PyMuPDF

     |████████████████████████████████| 5.3MB 2.6MB/s 


In [0]:
import fitz

In [0]:
def arrayStemmer(textArray):
  lematized_wordArray=[]
  porter_wordArray=[]
  porter = PorterStemmer()
  for text in textArray:
    text = text.lower()
    porter_wordArray.append(porter.stem(text))
  return (porter_wordArray)

def textAnalyze(text):
  text=text.lower()
  # tokenized_word=word_tokenize(text)
  tokenizer = RegexpTokenizer(r'\w+')
  tokenized_word=tokenizer.tokenize(text)
  stop_words=set(stopwords.words("english"))
  filtered_sent=[]
  for w in tokenized_word:
    if w not in stop_words:
      filtered_sent.append(w)
  porter_wordArray=[]
  porter = PorterStemmer()
  for text in filtered_sent:
    text=text.lower()
    porter_wordArray.append(porter.stem(text))
  finalWords=[]
  for w in porter_wordArray:
    try:
      width = float(w)
    except ValueError:
      finalWords.append(w)
  return finalWords

def scoreGenie(bag_of_words, stemmed_array):
  final_score = 0
  frequency_score = 0
  match_score = 0
  match_array = []
  for word in stemmed_array:
    for w in bag_of_words:
      if word == w:
        match_score+=1
        match_array.append(w)
  frequency_score = len(set(match_array))
  final_score = frequency_score * match_score
  return final_score

In [0]:
profit_bag_of_words = ['Profit','Income','Margins','Gross','Margin','profits','Revenues', 'revenue']
redflag_bag_of_words = ['disputes', 'difficulty', 'serious', 'adverse', 'unexpected']
cashflow_bag_of_words = ['cash flows', 'cash flow', 'free cash flow increase']
revenue_bag_of_words = ['New revenue','Expansion','Acquisitions','Acquired','Growth']



test_bag_of_words = profit_bag_of_words #Change here for the required bag of words
bag_of_words = arrayStemmer(test_bag_of_words)
print(test_bag_of_words)

bag_of_words = list(set(bag_of_words))
print(bag_of_words)

['Profit', 'Income', 'Margins', 'Gross', 'Margin', 'profits', 'Revenues', 'revenue']
['margin', 'gross', 'profit', 'revenu', 'incom']


Filter by key phrase to only reserve the key information

In [0]:
pdf_document = 'DMPL AR2018.pdf'
# pdf_document = 'AR_2017.pdf'
# key_words = ['profit', 'Profit', 'income', 'Income', 'revenue', 'Revenue']
reason_words = ['due to', 'impacted by', 'because', 'as a result', 'was attributed to', 'resulting', 'resulted']

doc = fitz.open(pdf_document)

matchedPageArray = []
tmp_full_text=''

for current_page in range(len(doc)):
  page = doc.loadPage(current_page)

  for word in reason_words:
    if page.searchFor(word):
      tmp_full_text =  tmp_full_text + '\n' + page.getText('text').replace('ﬂ ', 'fl').replace('ff ', 'ff').replace('ﬁ ', 'fi')
      matchedPageArray.append(current_page)

matchedPageArray = list(set(matchedPageArray))

print(matchedPageArray)

with open(f"{pdf_document}_{reason_words}.txt", "w") as text_file:
    text_file.write(tmp_full_text)

[130, 132, 7, 8, 9, 10, 11, 140, 143, 144, 145, 146, 149, 150, 151, 154, 155, 156, 157, 31, 162, 166, 41, 169, 44, 46, 47, 48, 175, 176, 51, 178, 181, 54, 55, 56, 185, 195, 68, 71, 199, 202, 203, 205, 210, 214, 215, 216, 92, 221, 225, 230, 106, 107, 108, 109, 110, 235, 236, 239, 126]


In [0]:
import re

alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    #print ("Inside split")
    text = " " + text + "  "
    #print (text)
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    if "e.g." in text: text = text.replace("e.g.","e<prd>g<prd>") 
    if "i.e." in text: text = text.replace("i.e.","i<prd>e<prd>")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    #print (text)
    text = text.encode('ascii',errors='ignore').decode()
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]

    #remove excessive space within sentences
    sentences2 = [' '.join(s.split()) for s in sentences]

    return sentences, sentences2

In [0]:
sentences, sentences2 = split_into_sentences(tmp_full_text)

print(len(sentences))
print(len(sentences2))

with open('sentences.txt', 'w') as f:
    for item in sentences:
        f.write("%s\n\n" % item)

with open('sentences_with_proper_spacing.txt', 'w') as f:
    for item in sentences2:
        f.write("%s\n\n" % item)

1270
1270


In [0]:
filtered_sentences = []

i = 0
for sent in sentences2:
  for key in reason_words:
    if sent.lower().find(key) > -1:
      # if i > 0:
      #   filtered_sentences.append(sentences2[i-1])
      filtered_sentences.append(sent)
      # if i+1 < len(sentences2):
      #   filtered_sentences.append(sentences2[i+1])
  i = i + 1

filtered_sentences = list(set(filtered_sentences))

print(len(filtered_sentences))

with open('sentences_with_focus', 'w') as f:
    for item in filtered_sentences:
        f.write("%s\n\n" % item)

101


filter with key financial figures

In [0]:
key_financial_figures = ['net profit', 'EBIDTA', 'gross profit', '']

In [0]:
! pip install news-corpus-builder

  Using cached https://files.pythonhosted.org/packages/5d/e1/43b98c242ce342d30fd574751513dbb6f75137503893e30813abec278ae5/news-corpus-builder-0.1.5.zip
  Using cached https://files.pythonhosted.org/packages/ab/f3/2e89688e4e1d8786e542ea460431f24f1392d885e60303dc20d476a96424/goose-extractor-1.0.25.tar.gz
  Using cached https://files.pythonhosted.org/packages/91/d8/7d37fec71ff7c9dbcdd80d2b48bcdd86d6af502156fc93846fb0102cb2c4/feedparser-5.2.1.tar.bz2
  Using cached https://files.pythonhosted.org/packages/3b/d4/3b5c17f00cce85b9a1e6f91096e1cc8e8ede2e1be8e96b87ce1ed09e92c5/cssselect-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/40/f2/6c9f2f3e696ee6a1fb0e4d7850617e224ed2b0b1e872110abffeca2a09d4/BeautifulSoup-3.2.2.tar.gz
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [0]:
from news_corpus_builder import NewsCorpusGenerator

# Location to save generated corpus
import os
corpus_dir = os.getcwd()

# Save results to sqlite or  files per article 
ex = NewsCorpusGenerator(corpus_dir,'sqlite')

# Retrieve 50 links related to the search term dogs and assign a category of   Pet to the retrieved links
links = ex.google_news_search('dogs','Pet',50)

# Generate and save corpus
ex.generate_corpus(links)

ModuleNotFoundError: ignored

Start XLNet exploration

https://github.com/zihangdai/xlnet/blob/master/notebooks/colab_imdb_gpu.ipynb

https://github.com/zihangdai/xlnet



In [0]:
! pip install sentencepiece

     |████████████████████████████████| 1.0MB 9.2MB/s 


In [0]:
! wget https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
! unzip cased_L-24_H-1024_A-16.zip

--2019-11-21 16:18:54--  https://storage.googleapis.com/xlnet/released_models/cased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 2a00:1450:4013:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1338042341 (1.2G) [application/zip]
Saving to: ‘cased_L-24_H-1024_A-16.zip’

cased_L-24_H-1024_A 100%[===================>]   1.25G  53.0MB/s    in 22s     

2019-11-21 16:19:17 (57.6 MB/s) - ‘cased_L-24_H-1024_A-16.zip’ saved [1338042341/1338042341]

Archive:  cased_L-24_H-1024_A-16.zip
   creating: xlnet_cased_L-24_H-1024_A-16/
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.index  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.data-00000-of-00001  
  inflating: xlnet_cased_L-24_H-1024_A-16/spiece.model  
  inflating: xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt.meta  
  inflating: xlnet_cased_L-24_H-1024_A-16